# Práca s modelmi Mistral

## Úvod

V tejto lekcii sa dozviete:
- Aké rôzne modely Mistral existujú
- Kedy a na čo sa jednotlivé modely používajú
- Ukážky kódu, ktoré prezentujú jedinečné vlastnosti každého modelu


## Mistral modely

V tejto lekcii sa pozrieme na 3 rôzne modely Mistral:  
**Mistral Large**, **Mistral Small** a **Mistral Nemo**.

Každý z týchto modelov je dostupný zadarmo na trhu modelov Github. Kód v tomto zápisníku bude používať tieto modely na spúšťanie kódu. Viac informácií o používaní Github Models na [prototypovanie s AI modelmi](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) nájdete tu.


## Mistral Large 2 (2407)
Mistral Large 2 je momentálne vlajkový model od Mistral a je navrhnutý pre firemné použitie.

Model je vylepšením pôvodného Mistral Large a ponúka
- Väčšie kontextové okno – 128k oproti 32k
- Lepší výkon pri matematických a programátorských úlohách – priemerná presnosť 76,9 % oproti 60,4 %
- Zvýšený výkon v rôznych jazykoch – podporované jazyky: angličtina, francúzština, nemčina, španielčina, taliančina, portugalčina, holandčina, ruština, čínština, japončina, kórejčina, arabčina a hindčina.

Vďaka týmto vlastnostiam vyniká Mistral Large v:
- *Retrieval Augmented Generation (RAG)* – vďaka väčšiemu kontextovému oknu
- *Function Calling* – tento model má natívnu podporu volania funkcií, čo umožňuje integráciu s externými nástrojmi a API. Tieto volania je možné vykonávať paralelne alebo postupne za sebou.
- *Generovanie kódu* – tento model je špičkový pri generovaní kódu v jazykoch Python, Java, TypeScript a C++.


V tomto príklade používame Mistral Large 2 na spustenie RAG vzoru nad textovým dokumentom. Otázka je napísaná v kórejčine a pýta sa na autorove aktivity pred vysokou školou.

Na vytvorenie embeddingov textového dokumentu aj otázky sa používa Cohere Embeddings Model. V tomto príklade sa ako vektorový úložisko používa Python balík faiss.

Prompt, ktorý sa posiela modelu Mistral, obsahuje otázky aj získané úseky, ktoré sú podobné otázke. Model potom poskytne odpoveď v prirodzenom jazyku.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small je ďalší model z rodiny Mistral, ktorý patrí do kategórie premier/enterprise. Ako už názov napovedá, ide o malý jazykový model (SLM). Výhody používania Mistral Small sú:
- Úspora nákladov v porovnaní s Mistral LLM ako Mistral Large a NeMo – až 80% zníženie ceny
- Nízka latencia – rýchlejšie odpovede v porovnaní s LLM modelmi Mistral
- Flexibilita – dá sa nasadiť v rôznych prostrediach s menšími nárokmi na zdroje

Mistral Small je ideálny na:
- Úlohy založené na texte, ako je sumarizácia, analýza sentimentu a preklad
- Aplikácie, kde sa často posielajú požiadavky, vďaka jeho cenovej efektívnosti
- Úlohy s nízkou latenciou, ako je kontrola kódu a návrhy kódu


## Porovnanie Mistral Small a Mistral Large

Na zobrazenie rozdielov v latencii medzi Mistral Small a Large spustite nasledujúce bunky.

Mali by ste vidieť rozdiel v čase odozvy približne 3-5 sekúnd. Všimnite si tiež rozdiely v dĺžke a štýle odpovedí na rovnaký prompt.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

V porovnaní s ďalšími dvoma modelmi spomenutými v tejto lekcii je Mistral NeMo jediný voľne dostupný model s licenciou Apache2.

Považuje sa za vylepšenie predchádzajúceho open source LLM od Mistral, Mistral 7B.

Ďalšie vlastnosti modelu NeMo sú:

- *Efektívnejšia tokenizácia:* Tento model používa tokenizér Tekken namiesto bežnejšieho tiktoken. To umožňuje lepší výkon vo viacerých jazykoch a pri práci s kódom.

- *Doladiteľnosť:* Základný model je dostupný na doladenie. To poskytuje väčšiu flexibilitu v prípadoch použitia, kde je doladenie potrebné.

- *Natívne volanie funkcií* – Rovnako ako Mistral Large, aj tento model bol trénovaný na volanie funkcií. Vďaka tomu je jedinečný, pretože patrí medzi prvé open source modely s touto schopnosťou.


## Mistral NeMo

V porovnaní s ďalšími dvoma modelmi spomenutými v tejto lekcii je Mistral NeMo jediný voľne dostupný model s licenciou Apache2.

Považuje sa za vylepšenie predchádzajúceho open source LLM od Mistral, Mistral 7B.

Ďalšie vlastnosti modelu NeMo sú:

- *Efektívnejšia tokenizácia:* Tento model používa tokenizér Tekken namiesto bežnejšieho tiktoken. To umožňuje lepší výkon vo viacerých jazykoch a pri práci s kódom.

- *Doladiteľnosť:* Základný model je dostupný na doladenie. To poskytuje väčšiu flexibilitu pre prípady použitia, kde je doladenie potrebné.

- *Natívne volanie funkcií* – Rovnako ako Mistral Large, aj tento model bol trénovaný na volanie funkcií. Vďaka tomu je jedným z prvých open source modelov, ktoré túto schopnosť majú.


### Porovnanie tokenizérov

V tomto príklade sa pozrieme na to, ako Mistral NeMo spracováva tokenizáciu v porovnaní s Mistral Large.

Oba príklady používajú rovnaký prompt, ale mali by ste si všimnúť, že NeMo vráti menej tokenov ako Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Učenie sa tu nekončí, pokračujte v ceste

Po dokončení tejto lekcie si pozrite našu [kolekciu o generatívnej AI](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) a pokračujte v rozširovaní svojich vedomostí o generatívnej umelej inteligencii!



---

**Vyhlásenie o vylúčení zodpovednosti**:  
Tento dokument bol preložený pomocou AI prekladateľskej služby [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, upozorňujeme, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Za autoritatívny zdroj sa považuje pôvodný dokument v jeho natívnom jazyku. Pre kritické informácie odporúčame profesionálny ľudský preklad. Nezodpovedáme za žiadne nedorozumenia alebo nesprávne interpretácie vzniknuté použitím tohto prekladu.
